In [1]:
 import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-27 18:32:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.53MB/s    in 0.2s    

2022-01-27 18:32:53 (5.53 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate() 

In [22]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').option("sep", "\t").csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [23]:
num_rows = df.count()

print(f"The number of rows in this dataset: {num_rows}")

The number of rows in this dataset: 1752932


In [24]:
'''
review_id_table creation
  review_id TEXT PRIMARY KEY NOT NULL,
  customer_id INTEGER,
  product_id TEXT,
  product_parent INTEGER,
  review_date DATE -- this should be in the formate yyyy-mm-dd
'''
from pyspark.sql.functions import col

review_id_table_df = df.select("review_id", 'customer_id', 'product_id', 'product_parent', 'review_date')

review_id_table_df = review_id_table_df.withColumn('review_date', col('review_date').cast('date'))

review_id_table_df.printSchema()

review_id_table_df.show(5)

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [26]:
'''
CREATE TABLE products (
  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
  product_title TEXT
);
'''
products_df = df.select('product_id', 'product_title').dropDuplicates()

# products_df = products_df.filter(col('product_id') != 'B00005JL44')

products_df.printSchema()

products_df.show(5)

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Y8QMQN6|Best Baby Teether...|
|B002WN2BY6|The First Years T...|
|B000AV7O4O|Boon Frog Pod Bat...|
|B00UFOBDNY|Kinsa Smart Stick...|
|B00XF7DZ34|Critter Piller Ki...|
+----------+--------------------+
only showing top 5 rows



In [18]:
# products_df.select('product_id', 'product_title').filter(col('product_id') == 'B00005JL44').show()

+----------+-------------+
|product_id|product_title|
+----------+-------------+
+----------+-------------+



In [37]:

'''
-- Customer table for first data set
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
  customer_count INT
);
'''


customers_df = df.select('customer_id').groupby('customer_id').count().withColumnRenamed('count', 'customer_count')

customers_df = customers_df.withColumn('customer_count', col('customer_count').cast('int'))

customers_df.printSchema()

customers_df.show(5)

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             2|
|   44524872|             1|
|   16711087|             1|
|   14116630|             3|
|   10548283|             1|
+-----------+--------------+
only showing top 5 rows



In [28]:
'''
CREATE TABLE vine_table (
  review_id TEXT PRIMARY KEY,
  star_rating INTEGER,
  helpful_votes INTEGER,
  total_votes INTEGER,
  vine TEXT
);
'''

vine_table_df = df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')

vine_table_df.printSchema()

vine_table_df.show(5)

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8EWA1OFT84NX|          5|            0|          0|   N|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|
| RL5ESX231LZ0B|          5|            0|          0|   N|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [29]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<RDS_endpoint>:5432/<postgresDBname>"
config = {"user":"<postgres_username>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [30]:
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [31]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [38]:
customers_df.write.jdbc(url=jdbc_url, table='customers_baby', mode=mode, properties=config)

In [39]:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)